In [2]:
import polars as pl
import pandas as pd

#### initial sports subs filter

In [29]:
random_df = pl.read_csv('~/sports-language-in-politics/data/processed/random_sample.csv')
sports_df = pl.read_csv('~/sports-language-in-politics/data/processed/sports_sample.csv')

sports_subs = set(sports_df['subreddit'].unique().to_list())
random_subs = set(random_df['subreddit'].unique().to_list())

In [31]:
random_df_no_sports = random_df.filter(~pl.col('subreddit').is_in(sports_subs))

In [32]:
random_df_no_sports.write_csv('/users/ujan/sports-language-in-politics/data/processed/random_sample_no_sports.csv', separator=",")

#### remove gaming subs

In [2]:
random_df = pl.read_csv('~/sports-language-in-politics/data/processed/random_sample_no_sports.csv')

In [8]:
gaming_df = pl.read_csv('~/sports-language-in-politics/data/raw/gaming_subs.csv', truncate_ragged_lines=True)

In [21]:
gaming_subs = []
for i in range(len(gaming_df)):
    gaming_subs.append(gaming_df[i]['Name;Link'].to_list()[0].split('/r/')[-1])

In [25]:
random_df_no_gaming = random_df.filter(~pl.col('subreddit').is_in(gaming_subs))

In [29]:
random_df_no_gaming.write_csv('/users/ujan/sports-language-in-politics/data/processed/random_sample_no_sports.csv', separator=",")

In [3]:
random_df = pd.read_csv('~/sports-language-in-politics/data/processed/random_sample_no_sports.csv', usecols=["id", "subreddit"]) 

/var/folders/by/rktr_w596p97pmt8_cbknvs80000gn/T/ipykernel_37911/3484189379.py:1: DtypeWarning: Columns (15,16) have mixed types. Specify dtype option on import or set low_memory=False.
  random_df = pd.read_csv('~/sports-language-in-politics/data/processed/random_sample_no_sports.csv', usecols=["id", "subreddit"])


In [5]:
random_df.groupby('subreddit').agg({"id":"count"}).sort_values("id",ascending=False).head(100)

,id
subreddit,
AskReddit,564493
memes,79795
funny,76766
teenagers,76658
worldnews,74015
...,...
Philippines,10711
BigBrother,10533
Animemes,10520


##### filtering random comments for classifier

In [85]:
import polars as pl
import re
import glob
from tqdm.auto import tqdm
import os

In [86]:
path = '/users/ujan/sports-language-in-politics/data/raw/reddit_random/'

In [87]:
comment_list = []
sub_list = []
id_list = []
sub_id_list = []

bar = tqdm(range(50)) 
with os.scandir(path) as it:
    for entry in it:
        if entry.is_file() and entry.name.endswith(".csv"):
            #print(entry.name)
            df = pl.read_csv(entry.path).drop_nulls(subset=['body', 'subreddit', 'id', 'subreddit_id'])
            comment_list.extend(df['body'].to_list())
            sub_list.extend(df['subreddit'].to_list())
            id_list.extend(df['id'].to_list())
            sub_id_list.extend(df['subreddit_id'].to_list())
            bar.update(1)

  0%|          | 0/50 [00:00<?, ?it/s]

In [88]:
new_comment_list = []
bar = tqdm(range(len(comment_list)))

for comment in comment_list:
    new_comment = comment.replace("'", '')
    new_comment = re.sub(r"[^a-zA-Z0-9]+", ' ', new_comment).lower()
    new_comment_list.append(new_comment)
    bar.update(1)

  0%|          | 0/5225822 [00:00<?, ?it/s]

In [89]:
random_df = pl.DataFrame({
    'id': id_list,
    'comments': new_comment_list,
    'subreddit': sub_list,
    'sub_id': sub_id_list
})

random_df.head()

id,comments,subreddit,sub_id
str,str,str,str
"""dkn63pv""","""anime avatar i…","""anime""","""t5_2qh22"""
"""dkqi2tv""","""i say remove t…","""europe""","""t5_2qh4j"""
"""dk3721q""","""last time i re…","""europe""","""t5_2qh4j"""
"""djq7kca""","""her name was c…","""4chan""","""t5_2qh4w"""
"""djwlcaq""","""ah ben oui mai…","""france""","""t5_2qhjz"""


In [90]:
random_df.write_csv('/users/ujan/sports-language-in-politics/data/processed/random_sample.csv', separator=",")